In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import wandb
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from config import num_epochs, learning_rate, wandb_config, model
from preprocessing import stock_df
from preprocessing import train_loader, val_loader, test_loader, label_scaler, test_dates, auto_arima_model

class StockPredictionModule(pl.LightningModule):
    def __init__(self, model, label_scaler, train_loader, val_loader, test_loader, test_dates):
        super().__init__()
        self.model = model
        self.label_scaler = label_scaler
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.test_dates = test_dates
        self.criterion = nn.MSELoss()
        # self.criterion = torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean', label_smoothing=0.0)

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        seqs, labels = batch
        y_pred = self(seqs)
        loss = self.criterion(y_pred, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        seqs, labels = batch
        y_pred = self(seqs)
        loss = self.criterion(y_pred, labels)
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        seqs, labels = batch
        y_pred = self(seqs)
        loss = self.criterion(y_pred, labels)
        self.log("test_loss", loss)
        return loss

    def on_test_epoch_end(self):
        predictions, actuals = [], []
        for seqs, labels in self.test_loader:
            seqs, labels = seqs.to(self.device), labels.to(self.device)
            output = self(seqs)
            predictions.extend(output.view(-1).detach().cpu().numpy())
            actuals.extend(labels.view(-1).detach().cpu().numpy())

        predictions_rescaled = list(self.label_scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten())
        actuals_rescaled = list(self.label_scaler.inverse_transform(np.array(actuals).reshape(-1, 1)).flatten())
        baseline_rescaled = [actuals_rescaled[0]] + actuals_rescaled[:-1]
        arima_predictions = auto_arima_model.predict(n_periods=len(actuals_rescaled))
        baseline_constant = [1.0] * len(predictions_rescaled)

        len_test_set = len(predictions)
        actual_closing_prices = stock_df['Close'].values[-(len_test_set):]

        fig, ax = plt.subplots(figsize=(15, 7))
        ax.plot(test_dates[-100:], actuals_rescaled[-100:], label='Actual Relative Difference', color='black', linestyle='-')
        ax.plot(test_dates[-100:], predictions_rescaled[-100:], label='Predicted Relative Difference', color='green', linestyle='-')
        ax.plot(test_dates[-100:], baseline_rescaled[-100:], label='Baseline_1', color='darkblue', linestyle='-')
        ax.plot(test_dates[-100:], baseline_constant[-100:], label='Baseline_2', color='steelblue', linestyle='-')
        # ax.plot(test_dates[-100:], arima_predictions[-100:], label='Baseline', color='orange', linestyle='-') 
        ax.set_title('Relative Difference Prediction')
        ax.set_xlabel('Date')
        ax.set_ylabel('Relative Difference')
        ax.legend()
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
        plt.tight_layout()
        plt.show()
        filename = "plot.png"
        fig.savefig(filename)
        wandb.log({"Relative Difference Prediction": wandb.Image(filename)})
        os.remove(filename)
        plt.close(fig)
        
        """
        # Use plotting with rebasing to visualize the predictions as prices
        rebase_period = 30
        predicted_prices = [actual_closing_prices[0]]
        for i, relative_change in enumerate(predictions_rescaled[1:], 1):
            if i % rebase_period == 0:
                predicted_prices.append(actual_closing_prices[i])
            else:
                predicted_prices.append(predicted_prices[-1] * relative_change)
        baseline_prices = [actual_closing_prices[0]] + list(actual_closing_prices[:-1])

        fig, ax = plt.subplots(figsize=(15, 7))
        ax.plot(test_dates, actual_closing_prices, label='Actual Price', color='black', linestyle='-')
        ax.plot(test_dates, predicted_prices, label='Predicted Price', color='green', linestyle='-')
        ax.plot(test_dates, baseline_prices, label='Baseline', color='blue', linestyle='-') 
        ax.set_title('Price predictions based on last price in validation set')
        ax.set_xlabel('Date')
        ax.set_ylabel('Stock Price')
        ax.legend()
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
        plt.tight_layout()
        plt.show()
        filename = "plot.png"
        fig.savefig(filename)
        wandb.log({"Stock Price Prediction": wandb.Image(filename)})
        os.remove(filename)
        plt.close(fig)
        """

        """
        # We use for the first value the actual closing price and then multiply the relative change for each following timestep
        actual_prices = [actual_closing_prices[0]]
        for i in range(1, len(actuals_rescaled)):
            actual_prices.append(actual_prices[i-1] * actuals_rescaled[i])
        prediction_prices = [actual_closing_prices[0]]
        for i in range(1, len(predictions_rescaled)):
            prediction_prices.append(prediction_prices[i-1] * predictions_rescaled[i])
        baseline_prices = [actual_prices[0]] + actual_prices[:-1]
        fig, ax = plt.subplots(figsize=(15, 7))
        ax.plot(test_dates, actual_prices, label='Actual Price', color='black', linestyle='-')
        ax.plot(test_dates, prediction_prices, label='Predicted Price', color='green', linestyle='-')
        ax.plot(test_dates, baseline_prices, label='Baseline', color='blue', linestyle='-')
        ax.set_title('Stock Price Prediction')
        ax.set_xlabel('Date')
        ax.set_ylabel('Stock Price')
        ax.legend()
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
        plt.tight_layout()
        plt.show()
        filename = "plot.png"
        fig.savefig(filename)
        wandb.log({"Stock Price Prediction": wandb.Image(filename)})
        os.remove(filename)
        plt.close(fig)
        """
        
        net_abs_dev = torch.tensor([abs(predictions_rescaled[i] - actuals_rescaled[i]) for i in range(len(actuals_rescaled))])
        baseline_abs_dev = torch.tensor([abs(baseline_rescaled[i] - actuals_rescaled[i]) for i in range(len(actuals_rescaled))])
        diff_pos = torch.relu(baseline_abs_dev - net_abs_dev).reshape(-1).tolist()
        diff_min = (-torch.relu(net_abs_dev - baseline_abs_dev)).reshape(-1).tolist()
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.set_title('Model vs baseline performance comparison on test samples')
        ax.hlines(0, xmin=0, xmax=len(actuals_rescaled), linestyles='dashed', colors='black')
        ax.bar(list(range(len(actuals_rescaled))), diff_pos, color='g', label='Model Wins', width=1.0)
        ax.bar(list(range(len(actuals_rescaled))), diff_min, color='r', label='Baseline Wins', width=1.0)
        ax.legend()
        ax.set_xlabel('Test Sample Index')
        ax.set_ylabel('Difference in Absolute Deviation')
        plt.show()
        filename = "comparison_plot.png"
        fig.savefig(filename)
        wandb.log({"Model vs Baseline Performance Comparison": wandb.Image(filename)})
        os.remove(filename)
        plt.close(fig)

        model_actual_dev = torch.tensor([predictions_rescaled[i] - actuals_rescaled[i] for i in range(len(actuals_rescaled))])
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.set_title('Model deviations from actuals')
        ax.hlines(0, xmin=0, xmax=len(actuals_rescaled), linestyles='dashed', colors='black')
        ax.bar(list(range(len(actuals_rescaled))), model_actual_dev, color='g', label='Model Wins', width=1.0)
        ax.set_xlabel('Test Sample Index')
        ax.set_ylabel('Deviation from actuals')
        plt.show()
        filename = "comparison_plot.png"
        fig.savefig(filename)
        wandb.log({"Model deviations from actuals": wandb.Image(filename)})
        os.remove(filename)
        plt.close(fig)

        baseline_actual_dev = torch.tensor([baseline_rescaled[i] - actuals_rescaled[i] for i in range(len(actuals_rescaled))])
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.set_title('Baseline deviations from actuals')
        ax.hlines(0, xmin=0, xmax=len(actuals_rescaled), linestyles='dashed', colors='black')
        ax.bar(list(range(len(actuals_rescaled))), baseline_actual_dev, color='b', label='Baseline Wins', width=1.0)
        ax.set_xlabel('Test Sample Index')
        ax.set_ylabel('Deviation from actuals')
        plt.show()
        filename = "comparison_plot.png"
        fig.savefig(filename)
        wandb.log({"Baseline deviations from actuals": wandb.Image(filename)})
        os.remove(filename)
        plt.close(fig)

        arima_actual_dev = torch.tensor([arima_predictions[i] - actuals_rescaled[i] for i in range(len(actuals_rescaled))])
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.set_title('Arima deviations from actuals')
        ax.hlines(0, xmin=0, xmax=len(actuals_rescaled), linestyles='dashed', colors='black')
        ax.bar(list(range(len(actuals_rescaled))), arima_actual_dev, color='orange', width=1.0)
        ax.set_xlabel('Test Sample Index')
        ax.set_ylabel('Deviation from actuals')
        plt.show()
        filename = "comparison_plot.png"
        fig.savefig(filename)
        wandb.log({"Arima deviations from actuals": wandb.Image(filename)})
        os.remove(filename)
        plt.close(fig)

        actuals_rescaled = np.array(actuals_rescaled)
        predictions_rescaled = np.array(predictions_rescaled)
        baseline_rescaled = np.array(baseline_rescaled)
        arima_predictions = np.array(arima_predictions)

        model_mse = mean_squared_error(actuals_rescaled, predictions_rescaled)
        model_rmse = np.sqrt(model_mse)
        model_mae = mean_absolute_error(actuals_rescaled, predictions_rescaled)
        model_r2 = r2_score(actuals_rescaled, predictions_rescaled)
        model_mape = np.mean(np.abs((actuals_rescaled - predictions_rescaled) / (actuals_rescaled + 1e-8)))
        
        baseline_mse = mean_squared_error(actuals_rescaled, baseline_rescaled)
        baseline_rmse = np.sqrt(baseline_mse)
        baseline_mae = mean_absolute_error(actuals_rescaled, baseline_rescaled)
        baseline_r2 = r2_score(actuals_rescaled, baseline_rescaled)
        baseline_mape = np.mean(np.abs((actuals_rescaled - baseline_rescaled) / (actuals_rescaled + 1e-8)))

        arima_mse = mean_squared_error(actuals_rescaled, arima_predictions)
        arima_rmse = np.sqrt(arima_mse)
        arima_mae = mean_absolute_error(actuals_rescaled, arima_predictions)
        arima_r2 = r2_score(actuals_rescaled, arima_predictions)
        arima_mape = np.mean(np.abs((actuals_rescaled - arima_predictions) / actuals_rescaled))

        model_metrics = {
            "mse": model_mse,
            "rmse": model_rmse,
            "mae": model_mae,
            "mape": model_mape,
            "r2": model_r2,
        }
        baseline_metrics = {
            "mse": baseline_mse,
            "rmse": baseline_rmse,
            "mae": baseline_mae,
            "mape": baseline_mape,
            "r2": baseline_r2,
        }
        arima_metrics = {
            "mse": arima_mse,
            "rmse": arima_rmse,
            "mae": arima_mae,
            "mape": arima_mape,
            "r2": arima_r2,
        }
        model_baseline_performance_metrics = {
            "mse": round((baseline_mse / model_mse - 1) * 100, 2),
            "rmse": round((baseline_rmse / model_rmse - 1) * 100, 2),
            "mae": round((baseline_mae / model_mae - 1) * 100, 2),
            "mape": round((baseline_mape / model_mape - 1) * 100, 2),
            "r2": round((model_r2 / baseline_r2 - 1) * 100, 2),
        }
        model_arima_performance_metrics = {
            "mse": round((arima_mse / model_mse - 1) * 100, 2),
            "rmse": round((arima_rmse / model_rmse - 1) * 100, 2),
            "mae": round((arima_mae / model_mae - 1) * 100, 2),
            "mape": round((arima_mape / model_mape - 1) * 100, 2),
            "r2": round((model_r2 / arima_r2 - 1) * 100, 2),
        }

        metrics_table = wandb.Table(columns=["metric", "model", "baseline", "arima", "model-baseline performance comparison [%]", 
                                             "model-arima performance comparison [%]"])
        for metric in model_metrics.keys():
            metrics_table.add_data(metric, model_metrics[metric], baseline_metrics[metric], arima_metrics[metric], 
                                   model_baseline_performance_metrics[metric], model_arima_performance_metrics[metric])
        wandb.log({"metrics": metrics_table})

def main():
    seed_value = 42
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)

    torch.set_float32_matmul_precision("medium")
    wandb_logger = WandbLogger(project="RNN_single_step_forecasts", log_model="all", config=wandb_config)
    
    module = StockPredictionModule(model=model, label_scaler=label_scaler, train_loader=train_loader, val_loader=val_loader, test_loader=test_loader, test_dates = test_dates)

    # Device agnostic initialization
    if torch.cuda.is_available():   # Check for GPU availability
        accelerator = "gpu"
        devices = 1
    elif hasattr(torch, 'has_mps') and torch.backends.mps.is_built():  # Check for MPS availability (Apple Silicon)
        accelerator = "mps"
        devices = 1
    else:
        accelerator = None  # Defaults to CPU
        devices = None  # Ignored for CPU

    trainer = Trainer(max_epochs=num_epochs, logger=wandb_logger, accelerator=accelerator, devices=devices, enable_checkpointing=True)
    trainer.fit(module, train_dataloaders=train_loader, val_dataloaders=val_loader)
    trainer.test(dataloaders=test_loader, ckpt_path="best")

    wandb.finish()

if __name__ == "__main__":
    main()



---
### Hyperparameter optimization with Optuna

In [ ]:
# Adjust optuna code below with these changes to log performance metrics:


import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

class StockPredictionModule(LightningModule):
    # Existing methods remain unchanged

    def test_step(self, batch, batch_idx):
        seqs, labels = batch
        y_pred = self(seqs)
        loss = self.criterion(y_pred, labels)
        self.log("test_loss", loss)
        return {"y_pred": y_pred, "labels": labels}

    def on_test_epoch_end(self):
        test_result = self.trainer.test(self, dataloaders=self.test_loader, verbose=False)
        predictions = np.concatenate([x['y_pred'].cpu().numpy() for x in test_result])
        actuals = np.concatenate([x['labels'].cpu().numpy() for x in test_result])

        # Assuming predictions and actuals are properly scaled or normalized if necessary
        mse = mean_squared_error(actuals, predictions)
        rmse = np.sqrt(mse)
        r2 = r2_score(actuals, predictions)

        self.log_dict({
            "model_mse": mse,
            "model_rmse": rmse,
            "model_r2": r2
        })

def objective(trial):
    # Configuration setup remains the same

    wandb.init(project="optuna_hyperparameter_tuning", entity="frederik135", reinit=True, config=wandb_config)
    model = GRU(**model_config).to(device)
    module = StockPredictionModule(model=model, label_scaler=label_scaler,
                                   train_loader=train_loader, val_loader=val_loader, test_loader=test_loader, test_dates=None)
    module.hparams.learning_rate = learning_rate

    # Trainer and training setup remains the same
    
    trainer.fit(module, train_dataloaders=train_loader, val_dataloaders=val_loader)
    trainer.test(module, dataloaders=test_loader)
    val_result = trainer.validate(module, dataloaders=val_loader, verbose=False)
    val_loss = val_result[0].get('val_loss', float('inf'))

    # Fetch the logged metrics for return to Optuna
    mse = module.logged_metrics.get('model_mse', float('inf'))
    rmse = module.logged_metrics.get('model_rmse', float('inf'))
    r2 = module.logged_metrics.get('model_r2', float('inf'))

    # Finish the W&B run and return a metric for Optuna to minimize
    wandb.log({"val_loss": val_loss, "mse": mse, "rmse": rmse, "r2": r2})
    wandb.finish()

    return val_loss  # Or return another metric such as mse if that's your focus

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
print("Best hyperparameters: ", study.best_trial.params)


In [1]:
import optuna
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
import torch
from models import LSTM, GRU, FCNN
from config import model_config, device, seq_length, architecture
from preprocessing import train_loader, val_loader, label_scaler
import wandb

class StockPredictionModule(LightningModule):
    def __init__(self, model, label_scaler, train_loader, val_loader, test_loader, test_dates):
        super().__init__()
        self.model = model
        self.label_scaler = label_scaler
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.test_dates = test_dates
        self.criterion = torch.nn.MSELoss()

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        seqs, labels = batch
        y_pred = self(seqs)
        loss = self.criterion(y_pred, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        seqs, labels = batch
        y_pred = self(seqs)
        loss = self.criterion(y_pred, labels)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"val_loss": loss}

    def test_step(self, batch, batch_idx):
        seqs, labels = batch
        y_pred = self(seqs)
        loss = self.criterion(y_pred, labels)
        self.log("test_loss", loss)
        return loss

def objective(trial):
    # Define hyperparameters to tune
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_size = trial.suggest_int('hidden_size', 16, 128)
    dropout_prob = trial.suggest_uniform('dropout_prob', 0.0, 0.5)

    # Update model configuration
    model_config.update({
        "hidden_layer_size": hidden_size,
        "num_layers": num_layers,
        "dropout_prob": dropout_prob
    })

    # Update wandb configuration for the current trial
    wandb_config = {
        "architecture": architecture,
        "num_units": hidden_size,  # Reflects the current trial's suggestion
        "num_layers": num_layers,  # Reflects the current trial's suggestion
        "dropout": dropout_prob,  # Reflects the current trial's suggestion
        "seq_length": seq_length,
        "epochs": 50,  # Reflect current setup
        "learning_rate": learning_rate  # Reflects the current trial's suggestion
    }

    wandb.init(project="optuna_hyperparameter_tuning", entity="frederik135", reinit=True, config=wandb_config)
    model = GRU(**model_config).to(device)
    module = StockPredictionModule(model=model, label_scaler=label_scaler,
                                   train_loader=train_loader, val_loader=val_loader, test_loader=None, test_dates=None)
    module.hparams.learning_rate = learning_rate

    # Device agnostic initialization
    if torch.cuda.is_available():
        accelerator = "gpu"
        devices = 1
    elif hasattr(torch, 'has_mps') and torch.backends.mps.is_built():
        accelerator = "mps"
        devices = 1
    else:
        accelerator = None
        devices = None

    wandb_logger = WandbLogger(project="optuna_hyperparameter_tuning", log_model="all", config=wandb_config)
    trainer = Trainer(
        logger=wandb_logger,
        max_epochs=70,
        callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=10)],
        accelerator=accelerator,
        devices=devices,
        enable_checkpointing=False,
        enable_progress_bar=False
    )
    
    trainer.fit(module, train_dataloaders=train_loader, val_dataloaders=val_loader)
    val_result = trainer.validate(module, dataloaders=val_loader, verbose=False)
    val_loss = val_result[0].get('val_loss', float('inf'))
    wandb.finish()
    return val_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("Best hyperparameters: ", study.best_trial.params)

c:\Users\janfr\anaconda3\envs\pytorch_3_10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[*********************100%%**********************]  1 of 1 completed
[I 2024-05-02 20:41:38,416] A new study created in memory with name: no-name-7b8c4611-ae36-46eb-aa83-041a2719691e
C:\Users\janfr\AppData\Local\Temp\ipykernel_14440\945958637.py:52: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
C:\Users\janfr\AppData\Local\Temp\ipykernel_14440\945958637.py:55: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/r

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
c:\Users\janfr\anaconda3\envs\pytorch_3_10\lib\site-packages\pytorch_lightning\loggers\wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | model     | GRU     | 4.8 K 
1 | criterion | MSELoss | 0     
----------

epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇███
train_loss_epoch,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,▃▁▇▂▃▃▁▃▁▆▁▃▁▃▃▃▄▁▄▃▃▁▃▂▂▆▁█▂▅▃▄▁▄▃▆▅▂▃▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
val_loss,▅▃▄▃▂▂▂▁▅▁▁█▂▃▂▃▁▅▅
epoch,18
train_loss_epoch,0.00483
train_loss_step,0.00244
trainer/global_step,2502
val_loss,0.00171


[I 2024-05-02 20:42:23,216] Trial 0 finished with value: 0.00170747225638479 and parameters: {'learning_rate': 0.021283024611366413, 'num_layers': 2, 'hidden_size': 20, 'dropout_prob': 0.09451885489639611}. Best is trial 0 with value: 0.00170747225638479.
C:\Users\janfr\AppData\Local\Temp\ipykernel_14440\945958637.py:52: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
C:\Users\janfr\AppData\Local\Temp\ipykernel_14440\945958637.py:55: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_prob = trial.suggest_uniform('dropout_prob', 0.0, 0.5)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\janfr\anaconda3\envs\pytorch_3_10\lib\site-packages\pytorch_lightning\loggers\wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | model     | GRU     | 39.3 K
1 | criterion | MSELoss | 0     
--------------------------------------
39.3 K    Trainable params
0         Non-trainable params
39.3 K    Total params
0.157     Total estimated model params size (MB)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

[I 2024-05-02 22:56:08,772] Trial 98 finished with value: 0.001657161395996809 and parameters: {'learning_rate': 9.796621474096517e-05, 'num_layers': 2, 'hidden_size': 125, 'dropout_prob': 0.3848602058727114}. Best is trial 29 with value: 0.0016484229126945138.
C:\Users\janfr\AppData\Local\Temp\ipykernel_14440\945958637.py:52: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
C:\Users\janfr\AppData\Local\Temp\ipykernel_14440\945958637.py:55: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_prob = trial.suggest_uniform('dropout_prob', 0.0, 0.5)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\janfr\anaconda3\envs\pytorch_3_10\lib\site-packages\pytorch_lightning\loggers\wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | model     | GRU     | 130 K 
1 | criterion | MSELoss | 0     
--------------------------------------
130 K     Trainable params
0         Non-trainable params
130 K     Total params
0.521     Total estimated model params size (MB)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇███
train_loss_epoch,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,▃▂▁▂▂▁▁▁▁▂▂▂▁▁▁▁▁▂▂▂▁▁▂▁▁▂▁▁▂█▁▂▁▂▅▂▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,29
train_loss_epoch,0.00489
train_loss_step,0.00122
trainer/global_step,4031
val_loss,0.00165


[I 2024-05-02 22:56:58,882] Trial 99 finished with value: 0.001653599552810192 and parameters: {'learning_rate': 2.128933509463325e-05, 'num_layers': 2, 'hidden_size': 117, 'dropout_prob': 0.34206600855197405}. Best is trial 29 with value: 0.0016484229126945138.


Best hyperparameters:  {'learning_rate': 0.00018887149084660608, 'num_layers': 2, 'hidden_size': 37, 'dropout_prob': 0.45255852589403345}


In [ ]:
print("Best hyperparameters: ", study.best_trial.params)